In [1]:
from __future__ import print_function
from math import ceil
import numpy as np
import sys
import pdb

import torch
import torch.optim as optim
import torch.nn as nn

import generator
import discriminator
import helpers
import pickle

In [2]:
CUDA = True
VOCAB_SIZE = 12028
MAX_SEQ_LEN = 30
START_LETTER = 0
BATCH_SIZE = 64
MLE_TRAIN_EPOCHS = 100
ADV_TRAIN_EPOCHS = 20
POS_NEG_SAMPLES = 10700 # 10847 

GEN_EMBEDDING_DIM = 200
GEN_HIDDEN_DIM = 64
DIS_EMBEDDING_DIM = 64
DIS_HIDDEN_DIM = 64
idx = 0

def decode(sentence, itov):
    return " ".join(itov[int(i)] for i in sentence)

def getbatch(data, s, e):
    temp = data.cpu()
    if len(temp) < e - s:
        return data
    start, end = s % len(temp), e % len(temp)
    if end < start:
        res = np.vstack([temp[start:], temp[:end]])
    else:
        res = temp[start:end]
    return torch.LongTensor(res)

def train_generator_MLE(gen, gen_opt, oracle, real_data_samples, epochs):
    """
    Max Likelihood Pretraining for the generator
    """
    print("epoch ", epochs)

    for epoch in range(epochs):
        print('epoch %d : ' % (epoch + 1), end='')
        sys.stdout.flush()
        total_loss = 0

        for i in range(0, POS_NEG_SAMPLES, BATCH_SIZE):
            inp, target = helpers.prepare_generator_batch(real_data_samples[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                          gpu=CUDA)
            gen_opt.zero_grad()
            loss = gen.batchNLLLoss(inp, target)
            loss.backward()
            gen_opt.step()

            total_loss += loss.data.item()

            if (i / BATCH_SIZE) % ceil(
                            ceil(POS_NEG_SAMPLES / float(BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                print('.', end='')
                sys.stdout.flush()

        # each loss in a batch is loss per sample
        total_loss = total_loss / ceil(POS_NEG_SAMPLES / float(BATCH_SIZE)) / MAX_SEQ_LEN

        # sample from generator and compute oracle NLL
        oracle_loss = helpers.batchwise_oracle_nll(gen, oracle, POS_NEG_SAMPLES, BATCH_SIZE, MAX_SEQ_LEN,
                                                   start_letter=START_LETTER, gpu=CUDA)

        print(' average_train_NLL = %.4f, oracle_sample_NLL = %.4f' % (total_loss, oracle_loss))


def train_generator_PG(gen, gen_opt, oracle, dis, num_batches):
    """
    The generator is trained using policy gradients, using the reward from the discriminator.
    Training is done for num_batches batches.
    """

    for batch in range(num_batches):
        s = gen.sample(BATCH_SIZE*2)        # 64 works best
        inp, target = helpers.prepare_generator_batch(s, start_letter=START_LETTER, gpu=CUDA)
        rewards = dis.batchClassify(target)

        gen_opt.zero_grad()
        pg_loss = gen.batchPGLoss(inp, target, rewards)
        pg_loss.backward()
        gen_opt.step()

    # sample from generator and compute oracle NLL
    oracle_loss = helpers.batchwise_oracle_nll(gen, oracle, POS_NEG_SAMPLES, BATCH_SIZE, MAX_SEQ_LEN,
                                                   start_letter=START_LETTER, gpu=CUDA)

    print(' oracle_sample_NLL = %.4f' % oracle_loss)


def train_discriminator(discriminator, dis_opt, real_data_samples, generator, oracle, d_steps, epochs):
    """
    Training the discriminator on real_data_samples (positive) and generated samples from generator (negative).
    Samples are drawn d_steps times, and the discriminator is trained for epochs epochs.
    """

    # generating a small validation set before training (using oracle and generator)
    pos_val = oracle.sample(100)
    neg_val = generator.sample(100)
    val_inp, val_target = helpers.prepare_discriminator_data(pos_val, neg_val, gpu=CUDA)

    for d_step in range(d_steps):
        s = helpers.batchwise_sample(generator, POS_NEG_SAMPLES, BATCH_SIZE)
        dis_inp, dis_target = helpers.prepare_discriminator_data(real_data_samples, s, gpu=CUDA)
        for epoch in range(epochs):
            print('d-step %d epoch %d : ' % (d_step + 1, epoch + 1), end='')
            sys.stdout.flush()
            total_loss = 0
            total_acc = 0

            for i in range(0, 2 * POS_NEG_SAMPLES, BATCH_SIZE):
                inp, target = dis_inp[i:i + BATCH_SIZE], dis_target[i:i + BATCH_SIZE]
                dis_opt.zero_grad()
                out = discriminator.batchClassify(inp)
                loss_fn = nn.BCELoss()
                loss = loss_fn(out, target)
                loss.backward()
                dis_opt.step()

                total_loss += loss.data.item()
                total_acc += torch.sum((out>0.5)==(target>0.5)).data.item()

                if (i / BATCH_SIZE) % ceil(ceil(2 * POS_NEG_SAMPLES / float(
                        BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                    print('.', end='')
                    sys.stdout.flush()

            total_loss /= ceil(2 * POS_NEG_SAMPLES / float(BATCH_SIZE))
            total_acc /= float(2 * POS_NEG_SAMPLES)

            val_pred = discriminator.batchClassify(val_inp)
            print(' average_loss = %.4f, train_acc = %.4f, val_acc = %.4f' % (
                total_loss, total_acc, torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/200.))


real_data = pickle.load(open('./mydata/real_data.txt', 'rb'))   # real_data
emb_mat = pickle.load(open('./mydata/word2vec.txt', 'rb'))    # embedding matrix
vocab_to_int = pickle.load(open('./mydata/word2idx.txt', 'rb')) # vocab to int
int_to_vocab = pickle.load(open('./mydata/idx2word.txt', 'rb')) # int to vocab


hidden_mat = torch.FloatTensor(np.random.rand(1, BATCH_SIZE, GEN_HIDDEN_DIM))
real_data = torch.LongTensor(real_data)
emb_mat = torch.tensor(emb_mat)
inp = torch.LongTensor([real_data[i][idx] for i in range(BATCH_SIZE)])
if CUDA:
    hidden_mat = hidden_mat.cuda()
    real_data = real_data.cuda()
    emb_mat = emb_mat.cuda()
    inp = inp.cuda()


oracle = generator.Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA, oracle_init=True)
gen = generator.Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
dis = discriminator.Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
oracle_samples = real_data
if CUDA:
    oracle = oracle.cuda()
    gen = gen.cuda()
    dis = dis.cuda()
    oracle_samples = oracle_samples.cuda()




C:\Users\dwkim\Desktop\seqGAN-master_학습모델 포함\generator.py:30: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(p, 0, 1)


In [4]:
model_dic = torch.load("seq30_b64_dim_200_v12028_mlep100_advp_20_posneg_10700.pth")

In [5]:
model_dic.keys()

dict_keys(['gen', 'dis', 'oracle', 'i2v', 'v2i'])

In [6]:
gen.load_state_dict(model_dic['gen'])
dis.load_state_dict(model_dic['dis'])
oracle.load_state_dict(model_dic['oracle'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
for i in gen.sample(10, start_letter=vocab_to_int["<start>"]):
    print(decode(i, int_to_vocab))

앙증맞은 스커트 와 캐 주얼 하고 시크 함 과 함께 구성 된 면 이 더욱 참 변별 력 있는 폭 , 발볼 너비 를 생략 해 주시 면 위치 에
드레스 업 아이템 이 가능하니까 , 가볍게 들기 좋은 가방 이에요 길이 로 일단 버핏 과 발목 마감 된 디자인 이라 더욱 시크 하게 힘 이 기대 되는 쇼츠 라서
페니 로퍼입니 다 채 , 내부 부터 정이 고 캐 주얼 하 게 활용 하시면 좋을 것 같구요 기대 이상 으로 요 하이 웨 이스트 디자인 과 발목 까지 정직하게
을 널찍하게 스타 일링 이 가능한 매력 적 인 발볼 이 매력 적 인 가방 이에요 다만 가방 이에요 따로 , 우아한 데 일리 로 , 서브 서브 서브 에코
수 있는 디자인 의 그래서 보통 원단 퀄리티 를 입더라도 보면 , 이런 효과 가 있는 가방 이에요 다만 살짝 부분 크로스 로 스트랩 입구 를 조이는 이용 해 주심
예요 캐 주얼 하고 , 비주 얼 이에요 수납 공간 과 더 감각 적 인 착 화 감도 예쁘셔요 캐 주얼 하 게 매치 해서 너무 피트 되지 않아서 오래
너무 기본 로프 을 잡아 주는 역시 베이지 사이즈 라 특히 나 유용하실 거 예요 다만 , 발볼 과 자연 스럽게 착용 하기 좋아요 ♥ 보기 에도 입체 적 인
넣어 계절 감 에도 자연 스럽게 소화 되는 팬츠 예요 쉽게 입문 템 백 이에요 장점 이에요 오히려 무조건 추천 해요 특유 , 숄 더크로스 가 예쁘게 담긴 소재 나
가벼운 만큼 두고두고 소지품 수납 가능하셔요 무리 없이 실용 성 긴 팔 이나 등 이나 하이 웨 이스트 타입 이라 가벼운 멋 이 어울리는 베이지 , 화이트 세 컬러 로
내 구도 로 잘 어울리거든요 캐 주얼 하 게 툭 주셔도 어렵지 않아서 오히려 데 일리 스타일 보다는 좀 아니구요 스타 일링 해 주시 면 참 예쁘답니다 다른 핏 경험
